**Continuation of TCGA-KIRC 1 and 2 notebooks. Based on `prepare bladder data for infino v2.ipynb`**

In `rcc make standata.ipynb`, we describe the procedure for creating a new list of marker gene transcripts (which also involved `R data prep.ipynb`). That list was saved as: `infino-rcc/bindea_and_our_own_marker_genes_lists_combined.pkl`

**Strategy:**

1. Extract counts and TPM from kallisto output --> done in `TCGA-KIRC 1` and `2`
1. Filter single origin sample **TPM** and bladder **TPM** to marker genes
2. Run ComBat and create training and testing datasets as in `rcc make standata.ipynb` (except we've already dropped problematic rows)
3. Run PCA on bladder and single origin together post-ComBat with different colors to make sure ComBat did its job.
4. Launch


# Filter to marker genes

In [1]:
import numpy as np
import matplotlib as mpl

import sys
sys.path.append('..')

import data
import models
import cache

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pystan
from time import time
from datetime import timedelta
import pickle
import dill

import os
if 'infino-rcc' in os.getcwd():
    os.chdir('..')
    print(os.getcwd())

INFO:stancache.seed:Setting seed to 1245502385
INFO:root:Setting CACHE_DIR = /home/jovyan/modelcache/mz
INFO:stancache.seed:Setting seed to 1245502385


/home/jovyan/work/model-single-origin-samples


In [2]:
sample_df = pickle.load(open('/modelcache/mz/prep_sample_df.cached.sample_n_82429761595.pkl', 'rb'))

In [3]:
sample_df.head()

,index,sample_id,filename,gene_name,est_counts,tpm,log1p_tpm,log1p_counts,CCR6+,CCR7+,...,log1p_tpm_rescaled_subset,log1p_tpm_rescaled,gene_cat,gene_id,B_cell,T_cell,new_gene_cat,new_gene_id,new_sample_cat,new_sample_id
0,0,1,ERR431566,A1BG,56.74329,6.931783,2.070878,4.056007,0.0,0.0,...,-3.517738,-5.478180,A1BG,1,0,1,A1BG,1,1,1
1,1,2,ERR431567,A1BG,71.32171,9.090650,2.311609,4.281124,0.0,0.0,...,-5.052464,-3.287291,A1BG,1,0,1,A1BG,1,2,2
2,2,3,ERR431568,A1BG,113.22460,8.845730,2.287038,4.738167,0.0,0.0,...,-3.303694,-3.510914,A1BG,1,1,0,A1BG,1,3,3
3,3,4,ERR431569,A1BG,131.51760,10.714400,2.460819,4.886715,0.0,0.0,...,-4.017633,-1.929339,A1BG,1,1,0,A1BG,1,4,4
4,4,5,ERR431570,A1BG,57.29375,6.973180,2.076083,4.065495,0.0,0.0,...,-4.708581,-5.430805,A1BG,1,0,1,A1BG,1,5,5


In [4]:
combined_marker_genes = pickle.load(open('infino-rcc/bindea_and_our_own_marker_genes_lists_combined.pkl', 'rb'))
len(combined_marker_genes)

817

In [5]:
test_df_counts = pd.read_csv('infino-rcc/tcga_kirc_data/tcga_kirc.2.subset.counts.filtered.tsv', sep='\t')
print(test_df_counts.shape)
test_df_counts.head()

(49878, 80)


,Gene_symbols,X953473f4.9927.4fd4.bfee.8f5908638cc7,df66c814.1407.4899.a6e3.9c2a928e6ea0,X2bdf69ff.b6fe.48c8.ac05.74ca195c6e1d,X87c1d789.9482.442a.8b42.97bb479060e3,X4ddfd589.5a62.4149.9ccc.6a539f0b37b7,X383ba5ca.1706.4959.b543.ea87fbd367bd,d9f44b98.58b7.4ebd.a6a6.8de472bf95a8,X550af27b.de50.402f.8a81.c508e3dc6781,c81f9d5b.f58d.411d.9e1a.f919a0ef5636,...,X14b8d86c.2fef.4c50.8633.a84e12d81b51,X0cc91a13.32d9.4262.a33d.264096b4102c,X9bae55b1.5a27.4a9f.b376.ad1ab4fc4ee2,X6b124b2e.dce9.48ef.bedb.7a2da9cd2ff8,X3b4be095.1d76.49de.8357.f932ba5d55bb,e644640b.78d3.460a.83fa.ea475620a3f0,X01f8736a.85bc.498d.91c6.1c053c479512,e40e21de.f116.4eb1.87af.3b16797ec3b2,X19de7074.f195.47f6.81fa.6d3bf01c1685,X0153168a.39ce.47d6.b7f5.ac1e0f312245
0,5S_rRNA,9.204280,11.249630,10.73302,8.732760,1.49641,6.319660,4.41094,9.926649,4.398960,...,5.938090,9.418420,6.698460,3.000000,8.251770,2.50000,3.544530,4.645700,2.095861,8.884160
1,7SK,1.435663,0.666666,0.00000,0.666666,0.00000,1.333334,0.00000,1.726206,0.666666,...,0.750000,1.333334,0.000000,0.666666,2.000000,1.89499,0.000000,1.333334,3.123140,0.658972
2,A1BG,128.138700,205.199900,215.17890,140.744100,7.04211,157.791700,104.51000,107.278100,64.783480,...,630.209400,292.625300,183.565400,97.333850,96.643000,133.19810,95.820200,88.380000,86.603100,218.628800
3,A1BG-AS1,137.141800,207.946790,187.30330,202.679170,2.32233,183.595830,41.12993,143.327220,76.198220,...,826.231000,249.736070,126.324510,53.022270,78.511700,137.71936,70.129900,103.813740,81.016800,306.343700
4,A1CF,6100.216000,12536.641166,1201.95240,2628.853730,1221.00020,2039.842842,2279.93500,4234.261301,4380.631751,...,21.391444,2514.811700,2348.325003,293.498800,2560.211761,4128.04890,3055.639354,798.055720,630.081400,454.589871


In [6]:
test_df_tpm = pd.read_csv('infino-rcc/tcga_kirc_data/tcga_kirc.2.subset.tpm.filtered.tsv', sep='\t')
print(test_df_tpm.shape)
test_df_tpm.head()

(49878, 80)


,Gene_symbols,X953473f4.9927.4fd4.bfee.8f5908638cc7,df66c814.1407.4899.a6e3.9c2a928e6ea0,X2bdf69ff.b6fe.48c8.ac05.74ca195c6e1d,X87c1d789.9482.442a.8b42.97bb479060e3,X4ddfd589.5a62.4149.9ccc.6a539f0b37b7,X383ba5ca.1706.4959.b543.ea87fbd367bd,d9f44b98.58b7.4ebd.a6a6.8de472bf95a8,X550af27b.de50.402f.8a81.c508e3dc6781,c81f9d5b.f58d.411d.9e1a.f919a0ef5636,...,X14b8d86c.2fef.4c50.8633.a84e12d81b51,X0cc91a13.32d9.4262.a33d.264096b4102c,X9bae55b1.5a27.4a9f.b376.ad1ab4fc4ee2,X6b124b2e.dce9.48ef.bedb.7a2da9cd2ff8,X3b4be095.1d76.49de.8357.f932ba5d55bb,e644640b.78d3.460a.83fa.ea475620a3f0,X01f8736a.85bc.498d.91c6.1c053c479512,e40e21de.f116.4eb1.87af.3b16797ec3b2,X19de7074.f195.47f6.81fa.6d3bf01c1685,X0153168a.39ce.47d6.b7f5.ac1e0f312245
0,5S_rRNA,9.914770,15.130672,25.363480,13.224230,3.869380,11.317250,4.362510,13.665296,4.233722,...,6.857678,13.437248,9.344670,5.197250,13.068880,4.319850,6.512180,9.392700,2.772839,17.498520
1,7SK,0.411977,0.117796,0.000000,0.150124,0.000000,0.344334,0.000000,0.394435,0.237672,...,0.157959,0.280088,0.000000,0.171198,0.489391,0.523055,0.000000,0.431138,1.700330,0.199897
2,A1BG,6.611734,7.657621,18.736413,5.590750,0.309164,8.832981,3.619899,4.672390,3.752960,...,30.057532,14.985862,7.511637,4.539710,4.106327,5.725944,4.635009,5.133269,7.756796,13.081544
3,A1BG-AS1,1.769861,2.154506,4.555689,2.742114,0.044404,2.860883,0.557519,1.780005,1.579688,...,9.587063,3.340660,2.063594,0.769476,1.055890,2.040633,1.381748,1.964373,1.997219,6.476828
4,A1CF,16.914616,27.108711,5.592854,7.241174,5.153873,6.403223,5.224765,11.907584,13.552498,...,0.053868,7.079731,6.363790,1.224351,6.665892,11.964710,10.849198,2.790016,3.062978,1.571386


In [7]:
# confirm gene lists identical (by design)
assert len(set(test_df_counts.Gene_symbols.unique()).symmetric_difference(set(test_df_tpm.Gene_symbols.unique()))) == 0

In [8]:
all_genes_train = sample_df.gene_name.unique()
all_genes_test = test_df_counts.Gene_symbols.unique()
all_genes_intersect = list(set(all_genes_train) & set(all_genes_test))
len(all_genes_intersect), len(all_genes_train), len(all_genes_test)

(29733, 34832, 49878)

In [9]:
# remove any marker transcripts that aren't present in our training+testing data
marker_genes_adjusted = list(set(combined_marker_genes) - (set(combined_marker_genes) - set(all_genes_intersect)))
len(all_genes_intersect), len(marker_genes_adjusted)

(29733, 740)

In [10]:
genes_to_sample_from = list(set(all_genes_intersect) - set(marker_genes_adjusted))
total_num_genes=900
n_genes_sample = total_num_genes-len(marker_genes_adjusted)
print('total number of genes we want:', total_num_genes)
print('desired number of housekeeping genes:', n_genes_sample)
sampled_genes = np.random.choice(genes_to_sample_from, size=n_genes_sample)
len(sampled_genes)

total number of genes we want: 900
desired number of housekeeping genes: 160


160

In [11]:
selected_genes = np.concatenate([marker_genes_adjusted, sampled_genes]).tolist()
assert len(selected_genes) == total_num_genes

In [12]:
# pickle out selected gene list
pickle.dump(selected_genes, open('infino-rcc/tcgakirc.selected_genes.pkl', 'wb'), pickle.HIGHEST_PROTOCOL)

In [13]:
selected_genes = pickle.load(open('infino-rcc/tcgakirc.selected_genes.pkl', 'rb'))

In [14]:
training_df = sample_df[sample_df['gene_name'].isin(selected_genes)].copy()
training_df.shape

(56700, 41)

In [15]:
# reset ids
training_df['new_gene_cat'] = training_df['gene_name'].astype('category')
training_df['new_gene_id'] = training_df['new_gene_cat'].cat.codes+1
training_df['new_sample_cat'] = training_df['sample_id'].astype('category')
training_df['new_sample_id'] = training_df['new_sample_cat'].cat.codes+1
training_df['new_gene_id'].describe()

count    56700.000000
mean       450.500000
std        259.809752
min          1.000000
25%        225.750000
50%        450.500000
75%        675.250000
max        900.000000
Name: new_gene_id, dtype: float64

In [16]:
test_df_counts = test_df_counts.loc[test_df_counts['Gene_symbols'].isin(selected_genes)]
test_df_tpm = test_df_tpm.loc[test_df_tpm['Gene_symbols'].isin(selected_genes)]
test_df_counts.shape, test_df_tpm.shape

((900, 80), (900, 80))

# Prepare to run ComBat

In [17]:
test_df_counts_log1p = test_df_counts.set_index('Gene_symbols').apply(np.log1p)
test_df_tpm_log1p = test_df_tpm.set_index('Gene_symbols').apply(np.log1p)
test_df_counts_log1p.head()

,X953473f4.9927.4fd4.bfee.8f5908638cc7,df66c814.1407.4899.a6e3.9c2a928e6ea0,X2bdf69ff.b6fe.48c8.ac05.74ca195c6e1d,X87c1d789.9482.442a.8b42.97bb479060e3,X4ddfd589.5a62.4149.9ccc.6a539f0b37b7,X383ba5ca.1706.4959.b543.ea87fbd367bd,d9f44b98.58b7.4ebd.a6a6.8de472bf95a8,X550af27b.de50.402f.8a81.c508e3dc6781,c81f9d5b.f58d.411d.9e1a.f919a0ef5636,cf0052ae.8440.49df.a20c.f260980c9922,...,X14b8d86c.2fef.4c50.8633.a84e12d81b51,X0cc91a13.32d9.4262.a33d.264096b4102c,X9bae55b1.5a27.4a9f.b376.ad1ab4fc4ee2,X6b124b2e.dce9.48ef.bedb.7a2da9cd2ff8,X3b4be095.1d76.49de.8357.f932ba5d55bb,e644640b.78d3.460a.83fa.ea475620a3f0,X01f8736a.85bc.498d.91c6.1c053c479512,e40e21de.f116.4eb1.87af.3b16797ec3b2,X19de7074.f195.47f6.81fa.6d3bf01c1685,X0153168a.39ce.47d6.b7f5.ac1e0f312245
Gene_symbols,,,,,,,,,,,,,,,,,,,,,
ABCB4,6.561507,6.104510,6.439970,5.885868,4.780072,6.648333,5.362536,6.329941,6.264012,5.965838,...,4.247332,5.312767,5.174255,4.715096,5.970627,5.919402,6.916850,5.997696,4.225654,4.939907
ABCC4,8.413711,9.095101,7.469923,8.347997,8.195988,8.481975,7.288892,8.980409,9.063691,8.632075,...,9.709272,8.689139,7.750338,8.324295,8.464965,7.820013,7.593914,7.647078,7.749776,7.222652
ABCG2,7.782389,8.377472,6.287859,7.632401,6.747587,7.241366,7.850105,6.456769,7.680635,5.081402,...,3.931826,6.428106,6.941189,5.262690,5.988962,8.043342,6.742881,6.579252,5.159055,7.412160
ABHD2,9.248136,9.730278,8.497598,9.209793,8.009982,9.039929,8.152356,9.553096,9.588735,9.443778,...,10.515211,8.849146,8.731848,8.734675,8.103957,8.759076,8.610271,7.867093,7.846059,8.823254
ABHD5,6.418562,6.963494,5.755742,6.629616,6.272879,6.818446,5.843869,6.260461,6.382567,6.234410,...,7.221956,6.985644,6.077690,6.235607,5.571532,5.870457,5.967761,5.908083,5.328380,5.902674


In [18]:
training_df.head()

,index,sample_id,filename,gene_name,est_counts,tpm,log1p_tpm,log1p_counts,CCR6+,CCR7+,...,log1p_tpm_rescaled_subset,log1p_tpm_rescaled,gene_cat,gene_id,B_cell,T_cell,new_gene_cat,new_gene_id,new_sample_cat,new_sample_id
4347,4347,1,ERR431566,ABCB4,1.000000,0.851882,0.616202,0.693147,0.0,0.0,...,1.998440,-0.188314,ABCB4,70,0,1,ABCB4,1,1,1
4348,4348,2,ERR431567,ABCB4,2.000000,0.397441,0.334642,1.098612,0.0,0.0,...,-0.115750,-0.636778,ABCB4,70,0,1,ABCB4,1,2,2
4349,4349,3,ERR431568,ABCB4,782.000170,28.925644,3.398716,6.663133,0.0,0.0,...,-6.838006,4.243620,ABCB4,70,1,0,ABCB4,1,3,3
4350,4350,4,ERR431569,ABCB4,139.999971,5.299960,1.840543,4.948760,0.0,0.0,...,-3.955303,1.761793,ABCB4,70,1,0,ABCB4,1,4,4
4351,4351,5,ERR431570,ABCB4,3.007070,0.197082,0.179887,1.388060,0.0,0.0,...,-2.390764,-0.883270,ABCB4,70,0,1,ABCB4,1,5,5


In [19]:
training_df['SubSet_and_sample'] = training_df['SubSet'].astype(str) + '.sampleid.' + training_df['new_sample_id'].astype(str)
pivoted_train_log1p_counts = pd.pivot_table(training_df,
                              index='gene_name',
                              columns='SubSet_and_sample',
                              values='log1p_counts'
                             )
pivoted_train_log1p_tpm = pd.pivot_table(training_df,
                              index='gene_name',
                              columns='SubSet_and_sample',
                              values='log1p_tpm'
                             )

In [20]:
# extract the column names for each batch
single_origin_cols = pivoted_train_log1p_tpm.columns
cohort_cols = test_df_tpm_log1p.columns
single_origin_cols, cohort_cols

(Index(['B_CD5.sampleid.26', 'B_CD5.sampleid.3', 'B_CD5.sampleid.31',
        'B_CD5.sampleid.34', 'B_Memory.sampleid.29', 'B_Memory.sampleid.4',
        'B_Memory.sampleid.43', 'B_Memory.sampleid.48', 'B_Memory.sampleid.52',
        'B_Naive.sampleid.21', 'B_Naive.sampleid.46', 'B_Naive.sampleid.54',
        'B_Naive.sampleid.59', 'B_Naive.sampleid.7',
        'CD4_Central_Memory.sampleid.17', 'CD4_Central_Memory.sampleid.33',
        'CD4_Central_Memory.sampleid.49', 'CD4_Central_Memory.sampleid.55',
        'CD4_Central_Memory.sampleid.61', 'CD4_Effector_Memory.sampleid.10',
        'CD4_Effector_Memory.sampleid.11', 'CD4_Effector_Memory.sampleid.23',
        'CD4_Effector_Memory.sampleid.38', 'CD4_Effector_Memory.sampleid.51',
        'CD4_Naive.sampleid.15', 'CD4_Naive.sampleid.16',
        'CD4_Naive.sampleid.45', 'CD4_Naive.sampleid.6', 'CD4_Naive.sampleid.9',
        'CD4_Th1.sampleid.12', 'CD4_Th1.sampleid.19', 'CD4_Th1.sampleid.22',
        'CD4_Th1.sampleid.5', 'CD4_Th1.samp

In [21]:
combined_raw_counts = pd.merge(test_df_counts_log1p, pivoted_train_log1p_counts, left_index=True, right_index=True)
print(test_df_counts_log1p.shape, pivoted_train_log1p_counts.shape, combined_raw_counts.shape)
combined_raw_counts.index.name = 'ProbeName'
combined_raw_counts.head()

(900, 79) (900, 63) (900, 142)


,X953473f4.9927.4fd4.bfee.8f5908638cc7,df66c814.1407.4899.a6e3.9c2a928e6ea0,X2bdf69ff.b6fe.48c8.ac05.74ca195c6e1d,X87c1d789.9482.442a.8b42.97bb479060e3,X4ddfd589.5a62.4149.9ccc.6a539f0b37b7,X383ba5ca.1706.4959.b543.ea87fbd367bd,d9f44b98.58b7.4ebd.a6a6.8de472bf95a8,X550af27b.de50.402f.8a81.c508e3dc6781,c81f9d5b.f58d.411d.9e1a.f919a0ef5636,cf0052ae.8440.49df.a20c.f260980c9922,...,CD8_Effector.sampleid.2,CD8_Effector.sampleid.20,CD8_Effector.sampleid.25,CD8_Effector.sampleid.56,CD8_Effector.sampleid.8,CD8_Naive.sampleid.27,CD8_Naive.sampleid.40,CD8_Naive.sampleid.47,CD8_Naive.sampleid.58,CD8_Naive.sampleid.62
ProbeName,,,,,,,,,,,,,,,,,,,,,
ABCB4,6.561507,6.104510,6.439970,5.885868,4.780072,6.648333,5.362536,6.329941,6.264012,5.965838,...,1.098612,2.079442,1.947962,2.302585,0.693147,2.198396,2.565462,2.080656,1.610132,2.079442
ABCC4,8.413711,9.095101,7.469923,8.347997,8.195988,8.481975,7.288892,8.980409,9.063691,8.632075,...,4.282047,3.903095,5.008213,4.290876,4.958242,4.785497,3.976639,3.833354,4.486932,5.096698
ABCG2,7.782389,8.377472,6.287859,7.632401,6.747587,7.241366,7.850105,6.456769,7.680635,5.081402,...,1.098612,3.970292,3.610918,3.135493,2.833213,3.583519,4.700480,4.127135,3.295838,2.944439
ABHD2,9.248136,9.730278,8.497598,9.209793,8.009982,9.039929,8.152356,9.553096,9.588735,9.443778,...,6.460766,7.344138,6.955704,7.094800,6.673489,6.546612,7.468739,6.721209,6.847852,6.439608
ABHD5,6.418562,6.963494,5.755742,6.629616,6.272879,6.818446,5.843869,6.260461,6.382567,6.234410,...,4.330733,4.442652,4.110873,4.264087,4.094344,4.094345,3.807695,3.178054,3.931826,3.914728


In [22]:
combined_raw_tpm = pd.merge(test_df_tpm_log1p, pivoted_train_log1p_tpm, left_index=True, right_index=True)
print(test_df_tpm_log1p.shape, pivoted_train_log1p_tpm.shape, combined_raw_tpm.shape)
combined_raw_tpm.index.name = 'ProbeName'
combined_raw_tpm.head()

(900, 79) (900, 63) (900, 142)


,X953473f4.9927.4fd4.bfee.8f5908638cc7,df66c814.1407.4899.a6e3.9c2a928e6ea0,X2bdf69ff.b6fe.48c8.ac05.74ca195c6e1d,X87c1d789.9482.442a.8b42.97bb479060e3,X4ddfd589.5a62.4149.9ccc.6a539f0b37b7,X383ba5ca.1706.4959.b543.ea87fbd367bd,d9f44b98.58b7.4ebd.a6a6.8de472bf95a8,X550af27b.de50.402f.8a81.c508e3dc6781,c81f9d5b.f58d.411d.9e1a.f919a0ef5636,cf0052ae.8440.49df.a20c.f260980c9922,...,CD8_Effector.sampleid.2,CD8_Effector.sampleid.20,CD8_Effector.sampleid.25,CD8_Effector.sampleid.56,CD8_Effector.sampleid.8,CD8_Naive.sampleid.27,CD8_Naive.sampleid.40,CD8_Naive.sampleid.47,CD8_Naive.sampleid.58,CD8_Naive.sampleid.62
ProbeName,,,,,,,,,,,,,,,,,,,,,
ABCB4,1.786577,1.215977,1.960582,1.278563,1.026254,2.062886,1.382510,1.729957,1.589234,1.432772,...,0.334642,0.409428,0.183653,0.734390,0.171621,0.267101,0.409472,0.283302,0.097145,0.662777
ABCC4,3.052037,3.404190,2.542866,2.944869,3.268303,3.195574,1.894603,3.481405,3.755063,3.088673,...,1.740690,1.182309,1.748660,1.749194,1.815492,1.583284,1.066921,0.810437,1.432758,1.523973
ABCG2,2.938073,3.135846,1.851018,2.716802,2.220262,2.397592,2.665559,1.655075,2.826349,0.655081,...,0.096339,0.834348,1.149255,0.455862,0.363216,0.592473,1.235624,0.969733,0.483663,0.424996
ABHD2,3.365501,3.568185,3.046352,3.292031,2.663061,3.247669,2.177218,3.570614,3.833753,3.476872,...,3.785061,3.303301,2.875261,3.323798,2.652309,2.879672,3.365206,2.906016,2.907709,2.697088
ABHD5,2.427891,2.638941,2.156326,2.594299,2.692320,2.861580,1.739094,2.180282,2.518814,2.119694,...,2.670204,2.110597,1.902689,2.109573,2.179342,1.839330,1.414390,1.096676,1.861320,1.976365


In [23]:
# confirm no nulls -- remove assert and len prefix to extract implicated column names if assertion fails
assert len(combined_raw_counts.columns[pd.isnull(combined_raw_counts).sum() > 0]) == 0
assert len(combined_raw_tpm.columns[pd.isnull(combined_raw_tpm).sum() > 0]) == 0

In [25]:
# file 1 for ComBat
combined_raw_counts.to_csv('infino-rcc/tcgakirc.combined.raw.combat_input.counts.tsv', sep='\t')
combined_raw_tpm.to_csv('infino-rcc/tcgakirc.combined.raw.combat_input.tpm.tsv', sep='\t')

## make sample information file

In [26]:
cohort_batch = pd.DataFrame({'Array name': cohort_cols})
cohort_batch['Batch'] = 2
cohort_batch.head()

,Array name,Batch
0,X953473f4.9927.4fd4.bfee.8f5908638cc7,2
1,df66c814.1407.4899.a6e3.9c2a928e6ea0,2
2,X2bdf69ff.b6fe.48c8.ac05.74ca195c6e1d,2
3,X87c1d789.9482.442a.8b42.97bb479060e3,2
4,X4ddfd589.5a62.4149.9ccc.6a539f0b37b7,2


In [27]:
single_origin_batch = pd.DataFrame({'Array name': single_origin_cols})
single_origin_batch['Batch'] = 1
single_origin_batch.head()

,Array name,Batch
0,B_CD5.sampleid.26,1
1,B_CD5.sampleid.3,1
2,B_CD5.sampleid.31,1
3,B_CD5.sampleid.34,1
4,B_Memory.sampleid.29,1


In [28]:
all_sample_info = pd.concat([cohort_batch, single_origin_batch])
print(all_sample_info.shape)
print(all_sample_info.Batch.value_counts())
all_sample_info['Sample name'] = all_sample_info['Array name']
all_sample_info = all_sample_info.loc[:, ['Array name', 'Sample name', 'Batch']] # rearrange just in case
all_sample_info.head()

(142, 2)
2    79
1    63
Name: Batch, dtype: int64


,Array name,Sample name,Batch
0,X953473f4.9927.4fd4.bfee.8f5908638cc7,X953473f4.9927.4fd4.bfee.8f5908638cc7,2
1,df66c814.1407.4899.a6e3.9c2a928e6ea0,df66c814.1407.4899.a6e3.9c2a928e6ea0,2
2,X2bdf69ff.b6fe.48c8.ac05.74ca195c6e1d,X2bdf69ff.b6fe.48c8.ac05.74ca195c6e1d,2
3,X87c1d789.9482.442a.8b42.97bb479060e3,X87c1d789.9482.442a.8b42.97bb479060e3,2
4,X4ddfd589.5a62.4149.9ccc.6a539f0b37b7,X4ddfd589.5a62.4149.9ccc.6a539f0b37b7,2


In [29]:
all_sample_info.to_csv('infino-rcc/tcgakirc.sample_info_batches.combat_input.tsv', sep='\t', index=False)

# Run ComBat

Done in `TCGA-KIRC 4 Run ComBat.ipynb` notebook. PCA before and after is shown there, too.

# Load corrected values and exponentiate

In [30]:
combat_output_counts = pd.read_csv('infino-rcc/tcgakirc.combat_output.counts.tsv', sep='\t')
combat_output_tpm = pd.read_csv('infino-rcc/tcgakirc.combat_output.tpm.tsv', sep='\t')
combat_output_counts.head()

,X953473f4.9927.4fd4.bfee.8f5908638cc7,df66c814.1407.4899.a6e3.9c2a928e6ea0,X2bdf69ff.b6fe.48c8.ac05.74ca195c6e1d,X87c1d789.9482.442a.8b42.97bb479060e3,X4ddfd589.5a62.4149.9ccc.6a539f0b37b7,X383ba5ca.1706.4959.b543.ea87fbd367bd,d9f44b98.58b7.4ebd.a6a6.8de472bf95a8,X550af27b.de50.402f.8a81.c508e3dc6781,c81f9d5b.f58d.411d.9e1a.f919a0ef5636,cf0052ae.8440.49df.a20c.f260980c9922,...,CD8_Effector.sampleid.2,CD8_Effector.sampleid.20,CD8_Effector.sampleid.25,CD8_Effector.sampleid.56,CD8_Effector.sampleid.8,CD8_Naive.sampleid.27,CD8_Naive.sampleid.40,CD8_Naive.sampleid.47,CD8_Naive.sampleid.58,CD8_Naive.sampleid.62
ABCB4,5.604789,4.989663,5.441197,4.695368,3.206950,5.721657,3.990956,5.293097,5.204355,4.803009,...,3.045455,3.828179,3.723255,4.006253,2.721885,3.923108,4.216035,3.829148,3.453659,3.828179
ABCC4,6.723146,7.416427,5.762887,6.656285,6.501623,6.792601,5.578697,7.299733,7.384469,6.945320,...,6.410316,6.040199,7.119551,6.418939,7.070746,6.902028,6.112029,5.972084,6.610424,7.205973
ABCG2,6.101578,6.699815,4.599124,5.950794,5.061290,5.557687,6.169653,4.768931,5.999285,3.386272,...,3.231073,6.076638,5.720532,5.249430,4.949900,5.693383,6.800186,6.232055,5.408317,5.060114
ABHD2,8.297409,8.716740,7.644649,8.264061,7.220558,8.116327,7.344384,8.562640,8.593636,8.467564,...,7.483745,8.625189,8.123277,8.303010,7.758614,7.594671,8.786192,7.820276,7.983917,7.456407
ABHD5,5.445609,6.019959,4.747008,5.668057,5.292061,5.867080,4.839893,5.278973,5.407671,5.251516,...,5.544617,5.649848,5.337894,5.481953,5.322352,5.322353,5.052832,4.460813,5.169545,5.153469


In [31]:
combat_output_counts.shape

(900, 142)

In [32]:
combat_output_counts.describe()

,X953473f4.9927.4fd4.bfee.8f5908638cc7,df66c814.1407.4899.a6e3.9c2a928e6ea0,X2bdf69ff.b6fe.48c8.ac05.74ca195c6e1d,X87c1d789.9482.442a.8b42.97bb479060e3,X4ddfd589.5a62.4149.9ccc.6a539f0b37b7,X383ba5ca.1706.4959.b543.ea87fbd367bd,d9f44b98.58b7.4ebd.a6a6.8de472bf95a8,X550af27b.de50.402f.8a81.c508e3dc6781,c81f9d5b.f58d.411d.9e1a.f919a0ef5636,cf0052ae.8440.49df.a20c.f260980c9922,...,CD8_Effector.sampleid.2,CD8_Effector.sampleid.20,CD8_Effector.sampleid.25,CD8_Effector.sampleid.56,CD8_Effector.sampleid.8,CD8_Naive.sampleid.27,CD8_Naive.sampleid.40,CD8_Naive.sampleid.47,CD8_Naive.sampleid.58,CD8_Naive.sampleid.62
count,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,...,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000
mean,5.225500,5.593956,4.679717,5.517372,4.308833,5.343265,4.648263,5.305688,5.204240,5.356100,...,4.699433,5.464439,5.366999,5.810710,5.294760,4.930439,4.974777,4.877542,4.944777,4.939211
std,2.542453,2.616350,2.466029,2.503334,2.727410,2.493382,2.467800,2.536932,2.551166,2.435279,...,2.348580,2.376212,2.590956,2.118447,2.541979,2.509687,2.578924,2.503913,2.448534,2.607432
min,-0.601396,-0.344818,-0.562608,-0.609586,-1.604174,-0.578340,-0.649708,-0.422892,-0.423921,-0.107256,...,-0.285562,-0.197449,-0.887762,-0.459678,-0.459678,-0.241122,-0.453357,-0.418825,-0.453357,-0.887762
25%,3.598187,3.951919,3.081598,4.058520,1.953967,3.922753,2.904010,3.741660,3.583875,3.816684,...,3.194402,4.074219,3.670891,4.770627,3.478867,3.156217,3.085942,3.215449,3.283113,3.087806
50%,5.694761,6.097771,5.069787,5.953220,4.526278,5.860035,5.187832,5.917613,5.801557,5.825126,...,5.117267,5.964332,5.902906,6.274942,5.823063,5.320128,5.346794,5.245195,5.354794,5.409009
75%,7.216736,7.706026,6.652118,7.486979,6.475770,7.312470,6.656430,7.232208,7.218129,7.195161,...,6.569283,7.312682,7.487353,7.311775,7.348365,6.946547,7.126563,6.935408,6.915119,7.090236
max,10.797200,10.562137,10.412852,10.455643,10.463155,10.487673,9.884699,10.583364,10.240961,12.093059,...,9.256282,10.474259,10.762238,10.410330,10.636382,10.735853,11.042533,10.452145,10.789398,10.699343


In [33]:
# exponentiate
batch_corrected_counts = combat_output_counts.apply(np.exp)
batch_corrected_tpm = combat_output_tpm.apply(np.exp)
batch_corrected_counts.describe()

,X953473f4.9927.4fd4.bfee.8f5908638cc7,df66c814.1407.4899.a6e3.9c2a928e6ea0,X2bdf69ff.b6fe.48c8.ac05.74ca195c6e1d,X87c1d789.9482.442a.8b42.97bb479060e3,X4ddfd589.5a62.4149.9ccc.6a539f0b37b7,X383ba5ca.1706.4959.b543.ea87fbd367bd,d9f44b98.58b7.4ebd.a6a6.8de472bf95a8,X550af27b.de50.402f.8a81.c508e3dc6781,c81f9d5b.f58d.411d.9e1a.f919a0ef5636,cf0052ae.8440.49df.a20c.f260980c9922,...,CD8_Effector.sampleid.2,CD8_Effector.sampleid.20,CD8_Effector.sampleid.25,CD8_Effector.sampleid.56,CD8_Effector.sampleid.8,CD8_Naive.sampleid.27,CD8_Naive.sampleid.40,CD8_Naive.sampleid.47,CD8_Naive.sampleid.58,CD8_Naive.sampleid.62
count,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,...,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000
mean,1254.533980,1814.603845,780.175103,1446.654069,986.507633,1232.324898,664.985031,1247.238477,1177.216487,1738.857006,...,626.760807,1252.327774,1420.743040,1252.577211,1265.957328,1039.660490,1091.234611,916.269872,929.744209,1074.444233
std,2811.134086,3451.976535,2238.436478,2731.699620,2713.992759,2427.228811,1377.518198,2513.193712,2324.026540,9633.185147,...,1201.947019,2500.317700,2856.587494,2186.208727,2548.184928,2615.203534,2781.128096,1999.159231,2403.555375,2499.270210
min,0.548046,0.708350,0.569721,0.543576,0.201056,0.560829,0.522198,0.655149,0.654476,0.898295,...,0.751592,0.820822,0.411576,0.631487,0.631487,0.785746,0.635491,0.657819,0.635491,0.411576
25%,36.533780,52.035263,21.794908,57.888842,7.056647,50.539526,18.248215,42.169250,36.012862,45.453747,...,24.396603,58.804748,39.287328,117.993219,32.423489,23.481897,21.888658,24.915260,26.659918,21.931058
50%,297.305661,444.865143,159.144474,384.991670,92.414841,350.737484,179.079938,371.523731,330.814972,338.704174,...,166.881342,389.292803,366.103267,531.095800,338.007419,204.410563,209.934387,189.653480,211.623345,223.411093
75%,1362.043620,2221.712031,774.422710,1784.653452,649.223422,1499.026674,777.772538,1383.276020,1363.954533,1332.966279,...,712.864121,1499.192509,1785.349533,1497.835210,1553.654914,1039.554445,1244.621127,1028.041509,1007.390978,1200.192452
max,48883.729250,38643.615155,33284.668152,34739.876432,35001.815403,35870.574107,19627.739618,39472.679945,28028.061287,178627.772183,...,10470.133078,35392.619824,47204.177683,33200.809120,41621.893408,45975.016213,62475.689220,34618.561922,48503.824317,44326.726783


In [34]:
assert all(batch_corrected_counts > 0)
assert all(batch_corrected_tpm > 0)

In [35]:
batch_corrected_counts.to_csv('infino-rcc/tcgakirc.plus-singleorigin.counts.batch_corrected.tsv', sep='\t')
batch_corrected_tpm.to_csv('infino-rcc/tcgakirc.plus-singleorigin.tpm.batch_corrected.tsv', sep='\t')

# Split into training and testing datasets to form Stan runs

In [36]:
batch_corrected_tpm.head()

,X953473f4.9927.4fd4.bfee.8f5908638cc7,df66c814.1407.4899.a6e3.9c2a928e6ea0,X2bdf69ff.b6fe.48c8.ac05.74ca195c6e1d,X87c1d789.9482.442a.8b42.97bb479060e3,X4ddfd589.5a62.4149.9ccc.6a539f0b37b7,X383ba5ca.1706.4959.b543.ea87fbd367bd,d9f44b98.58b7.4ebd.a6a6.8de472bf95a8,X550af27b.de50.402f.8a81.c508e3dc6781,c81f9d5b.f58d.411d.9e1a.f919a0ef5636,cf0052ae.8440.49df.a20c.f260980c9922,...,CD8_Effector.sampleid.2,CD8_Effector.sampleid.20,CD8_Effector.sampleid.25,CD8_Effector.sampleid.56,CD8_Effector.sampleid.8,CD8_Naive.sampleid.27,CD8_Naive.sampleid.40,CD8_Naive.sampleid.47,CD8_Naive.sampleid.58,CD8_Naive.sampleid.62
ABCB4,6.132064,2.804926,7.783835,3.056185,2.162606,8.955646,3.524200,5.674138,4.678708,3.775569,...,1.919938,2.037311,1.703170,2.636545,1.686988,1.819757,2.037381,1.843299,1.590187,2.490911
ABCC4,11.497252,15.794061,7.264524,10.438258,13.972745,13.085842,4.049046,16.932871,21.671690,11.883387,...,12.526329,6.477029,12.644819,12.652790,13.683508,10.401006,5.651743,4.174520,8.706744,9.697253
ABCG2,8.901503,10.482242,3.624627,7.413748,4.918174,5.694519,7.106305,3.082687,8.116328,1.348902,...,1.751195,5.436474,8.815472,3.040920,2.637813,3.750379,10.065195,6.692215,3.173498,2.900208
ABHD2,29.926762,35.320411,23.053732,28.182072,16.851865,27.178261,11.327711,35.390610,43.885329,32.779586,...,54.414951,25.004480,12.530635,25.845545,8.743583,12.620173,27.631923,13.168362,13.204393,9.398938
ABHD5,10.481962,13.037807,7.916075,12.449738,13.777537,16.412296,5.142477,8.114578,11.515093,7.621882,...,15.669661,9.155207,7.498200,9.146203,9.780008,7.055579,4.691434,3.457789,7.206154,8.047937


In [37]:
# extract columns in training set
single_origin_corrections_counts = batch_corrected_counts[single_origin_cols]
single_origin_corrections_tpm = batch_corrected_tpm[single_origin_cols]
single_origin_corrections_counts.head()

,B_CD5.sampleid.26,B_CD5.sampleid.3,B_CD5.sampleid.31,B_CD5.sampleid.34,B_Memory.sampleid.29,B_Memory.sampleid.4,B_Memory.sampleid.43,B_Memory.sampleid.48,B_Memory.sampleid.52,B_Naive.sampleid.21,...,CD8_Effector.sampleid.2,CD8_Effector.sampleid.20,CD8_Effector.sampleid.25,CD8_Effector.sampleid.56,CD8_Effector.sampleid.8,CD8_Naive.sampleid.27,CD8_Naive.sampleid.40,CD8_Naive.sampleid.47,CD8_Naive.sampleid.58,CD8_Naive.sampleid.62
ABCB4,1996.463900,1783.028671,894.751436,1371.221289,424.493435,453.937718,1087.460971,1052.492393,317.249213,2820.729977,...,21.019595,45.978743,41.398947,54.940601,15.208961,50.557309,67.764243,46.023314,31.615869,45.978739
ABCC4,133.383296,107.210737,265.460036,141.186284,1082.829544,1373.263274,1063.116061,1115.301265,638.842814,213.115285,...,608.085830,419.976756,1235.895537,613.351975,1177.025372,994.288941,451.253410,392.322613,742.797719,1347.455032
ABCG2,75.165654,50.295542,182.251124,25.306785,33.654222,58.595936,41.982500,50.295542,16.933511,58.595936,...,25.306785,435.562388,305.067278,190.457758,141.160803,296.896265,898.014668,508.799851,223.255532,157.608463
ABHD2,1732.682819,1687.424955,1526.681612,2028.915185,1846.911317,1816.427034,7027.470266,7790.440344,1161.320949,3464.394467,...,1778.891041,5570.216338,3372.054132,4036.000976,2341.657489,1987.576600,6543.265603,2490.592895,2933.398023,1730.917197
ABHD5,249.670025,236.818182,73.254299,149.909264,271.653366,377.687279,960.849087,851.863885,211.279741,268.498534,...,255.856480,284.248342,208.074100,240.315583,204.865252,204.865413,156.464895,86.557860,175.834844,173.030744


In [38]:
training_df.head()

,index,sample_id,filename,gene_name,est_counts,tpm,log1p_tpm,log1p_counts,CCR6+,CCR7+,...,log1p_tpm_rescaled,gene_cat,gene_id,B_cell,T_cell,new_gene_cat,new_gene_id,new_sample_cat,new_sample_id,SubSet_and_sample
4347,4347,1,ERR431566,ABCB4,1.000000,0.851882,0.616202,0.693147,0.0,0.0,...,-0.188314,ABCB4,70,0,1,ABCB4,1,1,1,CD4_Th2.sampleid.1
4348,4348,2,ERR431567,ABCB4,2.000000,0.397441,0.334642,1.098612,0.0,0.0,...,-0.636778,ABCB4,70,0,1,ABCB4,1,2,2,CD8_Effector.sampleid.2
4349,4349,3,ERR431568,ABCB4,782.000170,28.925644,3.398716,6.663133,0.0,0.0,...,4.243620,ABCB4,70,1,0,ABCB4,1,3,3,B_CD5.sampleid.3
4350,4350,4,ERR431569,ABCB4,139.999971,5.299960,1.840543,4.948760,0.0,0.0,...,1.761793,ABCB4,70,1,0,ABCB4,1,4,4,B_Memory.sampleid.4
4351,4351,5,ERR431570,ABCB4,3.007070,0.197082,0.179887,1.388060,0.0,0.0,...,-0.883270,ABCB4,70,0,1,ABCB4,1,5,5,CD4_Th1.sampleid.5


In [39]:
# take training_df existing
# join by new_sample_id and gene_name
melted_singleorigin_corrections_counts = pd.melt(single_origin_corrections_counts.reset_index(),
                             id_vars='index',
                             var_name='SubSet_and_sample',
                             value_name='counts_corrected')
melted_singleorigin_corrections_tpm = pd.melt(single_origin_corrections_tpm.reset_index(),
                             id_vars='index',
                             var_name='SubSet_and_sample',
                             value_name='tpm_corrected')
melted_singleorigin_corrections_counts = melted_singleorigin_corrections_counts.rename(columns={'index': 'gene_name'})
melted_singleorigin_corrections_tpm = melted_singleorigin_corrections_tpm.rename(columns={'index': 'gene_name'})
melted_singleorigin_corrections_tpm.head()

,gene_name,SubSet_and_sample,tpm_corrected
0,ABCB4,B_CD5.sampleid.26,26.087430
1,ABCC4,B_CD5.sampleid.26,3.256069
2,ABCG2,B_CD5.sampleid.26,1.964039
3,ABHD2,B_CD5.sampleid.26,17.549267
4,ABHD5,B_CD5.sampleid.26,8.407926


In [40]:
# add as new columns
new_training_df = pd.merge(training_df, melted_singleorigin_corrections_counts, on=['gene_name', 'SubSet_and_sample'], how='left')
new_training_df = pd.merge(new_training_df, melted_singleorigin_corrections_tpm, on=['gene_name', 'SubSet_and_sample'], how='left')
print(training_df.shape, new_training_df.shape)
assert training_df.shape[0] == new_training_df.shape[0]
new_training_df.head()[['SubSet_and_sample', 'gene_name', 'est_counts', 'tpm', 'counts_corrected', 'tpm_corrected']]

(56700, 42) (56700, 44)


,SubSet_and_sample,gene_name,est_counts,tpm,counts_corrected,tpm_corrected
0,CD4_Th2.sampleid.1,ABCB4,1.000000,0.851882,15.208961,2.400542
1,CD8_Effector.sampleid.2,ABCB4,2.000000,0.397441,21.019595,1.919938
2,B_CD5.sampleid.3,ABCB4,782.000170,28.925644,1783.028671,21.833836
3,B_Memory.sampleid.4,ABCB4,139.999971,5.299960,453.937718,6.341653
4,CD4_Th1.sampleid.5,ABCB4,3.007070,0.197082,26.481363,1.698088


In [41]:
assert new_training_df.new_gene_id.nunique() == new_training_df.new_gene_id.max()

In [42]:
pickle.dump(new_training_df, open('infino-rcc/tcgakirc.new_training_df_with_corrections_cols.pkl', 'wb'))

In [43]:
# call models.prep_stan_data
# note that it sets:
# 'y': sample_df.est_counts.astype(int).values
# if want to replace, feed manually as kwarg (don't forget to cast to int -- since negative binomial distribution)

[
    stan_data_training_counts_raw, stan_data_training_tpm_raw,
    stan_data_training_counts_corrected, stan_data_training_tpm_corrected
] = [
    models.prep_stan_data(
        sample_df=new_training_df,
        test_df=None,
        by='SubSet',
        y=new_training_df[colname].astype(int).values,
        colname=colname
    )
    for colname in ['est_counts', 'tpm', 'counts_corrected', 'tpm_corrected']
]

In [44]:
for standatadict in [
        stan_data_training_counts_raw, stan_data_training_tpm_raw,
        stan_data_training_counts_corrected, stan_data_training_tpm_corrected
]:
    assert all(type(t) == np.int64 for t in standatadict['y'])
    print(standatadict['colname'])

est_counts
tpm
counts_corrected
tpm_corrected


In [45]:
stan_data_training_counts_raw.keys()

dict_keys(['N', 'G', 'S', 'C', 'gene', 'sample', 'x', 'y', 'cell_features', 'M', 'colname'])

## extract cohort columns

In [46]:
cohort_corrected_counts = batch_corrected_counts[cohort_cols]
cohort_corrected_tpm = batch_corrected_tpm[cohort_cols]
cohort_corrected_counts.shape

(900, 79)

In [47]:
cohort_corrected_counts.head()

,X953473f4.9927.4fd4.bfee.8f5908638cc7,df66c814.1407.4899.a6e3.9c2a928e6ea0,X2bdf69ff.b6fe.48c8.ac05.74ca195c6e1d,X87c1d789.9482.442a.8b42.97bb479060e3,X4ddfd589.5a62.4149.9ccc.6a539f0b37b7,X383ba5ca.1706.4959.b543.ea87fbd367bd,d9f44b98.58b7.4ebd.a6a6.8de472bf95a8,X550af27b.de50.402f.8a81.c508e3dc6781,c81f9d5b.f58d.411d.9e1a.f919a0ef5636,cf0052ae.8440.49df.a20c.f260980c9922,...,X14b8d86c.2fef.4c50.8633.a84e12d81b51,X0cc91a13.32d9.4262.a33d.264096b4102c,X9bae55b1.5a27.4a9f.b376.ad1ab4fc4ee2,X6b124b2e.dce9.48ef.bedb.7a2da9cd2ff8,X3b4be095.1d76.49de.8357.f932ba5d55bb,e644640b.78d3.460a.83fa.ea475620a3f0,X01f8736a.85bc.498d.91c6.1c053c479512,e40e21de.f116.4eb1.87af.3b16797ec3b2,X19de7074.f195.47f6.81fa.6d3bf01c1685,X0153168a.39ce.47d6.b7f5.ac1e0f312245
ABCB4,271.724524,146.886978,230.718236,109.439052,24.703625,305.410588,54.106583,198.958657,182.063470,121.876628,...,12.059764,50.600730,41.994015,22.634876,122.664761,114.492055,438.379423,127.216490,11.712959,30.633392
ABCC4,831.429172,1663.080801,318.265939,777.656770,666.222031,891.229022,264.726493,1479.904552,1610.772320,1038.279478,...,3106.700003,1100.346546,423.348771,759.126933,875.937192,454.449486,361.059298,381.127671,423.106482,247.472922
ABCG2,446.561761,812.255664,99.397253,384.058331,157.793904,259.222446,478.020154,117.793226,403.140287,29.555554,...,9.305463,114.447453,191.697509,35.464107,73.599907,580.514047,157.049135,133.228123,31.955271,307.780362
ABHD2,4013.460562,6104.244702,2089.435501,3881.827514,1367.252094,3348.699452,1547.480950,5232.475978,5397.201954,4757.911387,...,12081.302795,2836.704826,2561.585963,2567.891426,1483.693915,2622.970734,2304.556472,1207.471578,1185.583044,2773.538872
ABHD5,231.738420,411.561635,115.238971,289.471465,198.752713,353.216027,126.455767,196.168396,223.111443,190.855346,...,540.434445,421.282708,161.795912,191.096400,94.902846,130.049607,144.094925,135.310505,73.447862,134.541336


In [48]:
test_df_counts.head()

,Gene_symbols,X953473f4.9927.4fd4.bfee.8f5908638cc7,df66c814.1407.4899.a6e3.9c2a928e6ea0,X2bdf69ff.b6fe.48c8.ac05.74ca195c6e1d,X87c1d789.9482.442a.8b42.97bb479060e3,X4ddfd589.5a62.4149.9ccc.6a539f0b37b7,X383ba5ca.1706.4959.b543.ea87fbd367bd,d9f44b98.58b7.4ebd.a6a6.8de472bf95a8,X550af27b.de50.402f.8a81.c508e3dc6781,c81f9d5b.f58d.411d.9e1a.f919a0ef5636,...,X14b8d86c.2fef.4c50.8633.a84e12d81b51,X0cc91a13.32d9.4262.a33d.264096b4102c,X9bae55b1.5a27.4a9f.b376.ad1ab4fc4ee2,X6b124b2e.dce9.48ef.bedb.7a2da9cd2ff8,X3b4be095.1d76.49de.8357.f932ba5d55bb,e644640b.78d3.460a.83fa.ea475620a3f0,X01f8736a.85bc.498d.91c6.1c053c479512,e40e21de.f116.4eb1.87af.3b16797ec3b2,X19de7074.f195.47f6.81fa.6d3bf01c1685,X0153168a.39ce.47d6.b7f5.ac1e0f312245
76,ABCB4,706.33700,446.87304,625.38770,358.914900,118.112890,770.49687,212.26514,560.123426,524.32215,...,68.918632,201.910938,175.665020,110.61954,390.75120,371.18904,1008.13570,401.50031,67.419253,138.757250
89,ABCC4,4507.46215,8910.53015,1753.47137,4220.717535,3625.371412,4825.97577,1462.94769,7944.877950,8634.97037,...,16468.615420,5937.068360,2321.357970,4121.82838,4744.56236,2488.93732,1985.07173,2093.51727,2320.051640,1369.117680
110,ABCG2,2396.99700,4347.00272,537.00004,2062.999600,851.000260,1394.99960,2565.00448,635.999900,2164.99560,...,50.000000,618.000740,1032.999310,191.99994,398.00040,3111.99950,847.00007,719.00078,173.000001,1655.000178
136,ABHD2,10384.18660,16818.22951,4901.97921,9993.524850,3009.864000,8432.18246,3470.54752,14087.238540,14598.38364,...,36871.128370,6967.434770,6196.171419,6213.71428,3306.53000,6367.22714,5486.73707,2608.96686,2554.641760,6789.322640
139,ABHD5,612.12094,1056.32190,315.00000,756.191570,529.000980,913.56245,344.11214,522.460330,590.44428,...,1368.165060,1080.002470,435.020880,509.61072,261.83651,353.41108,389.63008,366.99986,205.103910,365.014830


In [49]:
[
    melted_cohort_counts_raw, melted_cohort_tpm_raw,
    melted_cohort_counts_corrected, melted_cohort_tpm_corrected
] = [
    pd.melt(
        df.reset_index(),
        id_vars='index',
        var_name='sample',
        value_name=value_name).rename(columns={
            'index': 'gene_name'
        })
    for (df, value_name) in zip([
        test_df_counts.rename(columns={'Gene_symbols': 'index'}).set_index('index'),
        test_df_tpm.rename(columns={'Gene_symbols': 'index'}).set_index('index'),
        cohort_corrected_counts,
        cohort_corrected_tpm
    ], ['counts_raw', 'tpm_raw', 'counts_corrected', 'tpm_corrected'])
]

melted_cohort_counts_corrected.head()

,gene_name,sample,counts_corrected
0,ABCB4,X953473f4.9927.4fd4.bfee.8f5908638cc7,271.724524
1,ABCC4,X953473f4.9927.4fd4.bfee.8f5908638cc7,831.429172
2,ABCG2,X953473f4.9927.4fd4.bfee.8f5908638cc7,446.561761
3,ABHD2,X953473f4.9927.4fd4.bfee.8f5908638cc7,4013.460562
4,ABHD5,X953473f4.9927.4fd4.bfee.8f5908638cc7,231.738420


In [50]:
melted_cohort_counts_raw.head()

,gene_name,sample,counts_raw
0,ABCB4,X953473f4.9927.4fd4.bfee.8f5908638cc7,706.33700
1,ABCC4,X953473f4.9927.4fd4.bfee.8f5908638cc7,4507.46215
2,ABCG2,X953473f4.9927.4fd4.bfee.8f5908638cc7,2396.99700
3,ABHD2,X953473f4.9927.4fd4.bfee.8f5908638cc7,10384.18660
4,ABHD5,X953473f4.9927.4fd4.bfee.8f5908638cc7,612.12094


In [51]:
# merge them all

all_melted_cohorts = [
    melted_cohort_counts_raw, melted_cohort_tpm_raw,
    melted_cohort_counts_corrected, melted_cohort_tpm_corrected
]

# merge one
#melted_cohort = pd.merge(melted_cohort_counts, melted_cohort_tpm, how='inner', on=['gene_name', 'sample'])
#assert melted_cohort.shape[0] == melted_cohort_counts.shape[0] == melted_cohort_counts.shape[0]


from functools import reduce
# https://stackoverflow.com/a/44338256/130164
#e.g. reduce(lambda left,right: left+right, [1,2,3,4,5]) adds (1+2)+3 ... etc
melted_cohort = reduce(lambda left, right: pd.merge(left, right, how='inner', on=['gene_name', 'sample']),
                        all_melted_cohorts)

for i in all_melted_cohorts:
    assert melted_cohort.shape[0] == i.shape[0], i.columns

melted_cohort.head()

,gene_name,sample,counts_raw,tpm_raw,counts_corrected,tpm_corrected
0,ABCB4,X953473f4.9927.4fd4.bfee.8f5908638cc7,706.33700,4.968986,271.724524,6.132064
1,ABCC4,X953473f4.9927.4fd4.bfee.8f5908638cc7,4507.46215,20.158407,831.429172,11.497252
2,ABCG2,X953473f4.9927.4fd4.bfee.8f5908638cc7,2396.99700,17.879440,446.561761,8.901503
3,ABHD2,X953473f4.9927.4fd4.bfee.8f5908638cc7,10384.18660,27.948001,4013.460562,29.926762
4,ABHD5,X953473f4.9927.4fd4.bfee.8f5908638cc7,612.12094,10.334948,231.738420,10.481962


In [52]:
gene_name_to_id_map = new_training_df[['gene_name', 'new_gene_id']].drop_duplicates()
print(gene_name_to_id_map.shape, gene_name_to_id_map.new_gene_id.max())
gene_name_to_id_map.head()

(900, 2) 900


,gene_name,new_gene_id
0,ABCB4,1
63,ABCC4,2
126,ABCG2,3
189,ABHD2,4
252,ABHD5,5


In [53]:
# gene2 ids must correspond to gene ids (per model 6.2 inline comments)
# so don't do this:
# melted_rcc['new_gene_cat'] = melted_rcc['gene_name'].astype('category')
# melted_rcc['new_gene_id'] = melted_rcc['new_gene_cat'].cat.codes+1

# instead do:
melted_cohort2 = pd.merge(melted_cohort, gene_name_to_id_map, how='left', on='gene_name')
assert not any(pd.isnull(melted_cohort2['new_gene_id']))
melted_cohort2.head()

,gene_name,sample,counts_raw,tpm_raw,counts_corrected,tpm_corrected,new_gene_id
0,ABCB4,X953473f4.9927.4fd4.bfee.8f5908638cc7,706.33700,4.968986,271.724524,6.132064,1
1,ABCC4,X953473f4.9927.4fd4.bfee.8f5908638cc7,4507.46215,20.158407,831.429172,11.497252,2
2,ABCG2,X953473f4.9927.4fd4.bfee.8f5908638cc7,2396.99700,17.879440,446.561761,8.901503,3
3,ABHD2,X953473f4.9927.4fd4.bfee.8f5908638cc7,10384.18660,27.948001,4013.460562,29.926762,4
4,ABHD5,X953473f4.9927.4fd4.bfee.8f5908638cc7,612.12094,10.334948,231.738420,10.481962,5


In [54]:
# sample2 ids don't correspond to sample ids
melted_cohort2['new_sample_cat'] = melted_cohort2['sample'].astype('category')
melted_cohort2['new_sample_id'] = melted_cohort2['new_sample_cat'].cat.codes+1
melted_cohort2['new_sample_id'].describe()

count    71100.000000
mean        40.000000
std         22.803669
min          1.000000
25%         20.000000
50%         40.000000
75%         60.000000
max         79.000000
Name: new_sample_id, dtype: float64

# Pickle out stan data dicts

In [55]:
for standict, fname in zip([
        stan_data_training_counts_raw, stan_data_training_tpm_raw,
        stan_data_training_counts_corrected, stan_data_training_tpm_corrected
], [
        'infino-rcc/tcgakirc.singleorigin.stan_data.counts_raw.pkl',
        'infino-rcc/tcgakirc.singleorigin.stan_data.tpm_raw.pkl',
        'infino-rcc/tcgakirc.singleorigin.stan_data.counts_corrected.pkl',
        'infino-rcc/tcgakirc.singleorigin.stan_data.tpm_corrected.pkl'
]):
    print(standict.keys(), standict['colname'], fname)
    pickle.dump(standict, open(fname, 'wb'))

dict_keys(['N', 'G', 'S', 'C', 'gene', 'sample', 'x', 'y', 'cell_features', 'M', 'colname']) est_counts infino-rcc/tcgakirc.singleorigin.stan_data.counts_raw.pkl
dict_keys(['N', 'G', 'S', 'C', 'gene', 'sample', 'x', 'y', 'cell_features', 'M', 'colname']) tpm infino-rcc/tcgakirc.singleorigin.stan_data.tpm_raw.pkl
dict_keys(['N', 'G', 'S', 'C', 'gene', 'sample', 'x', 'y', 'cell_features', 'M', 'colname']) counts_corrected infino-rcc/tcgakirc.singleorigin.stan_data.counts_corrected.pkl
dict_keys(['N', 'G', 'S', 'C', 'gene', 'sample', 'x', 'y', 'cell_features', 'M', 'colname']) tpm_corrected infino-rcc/tcgakirc.singleorigin.stan_data.tpm_corrected.pkl


In [56]:
pickle.dump(melted_cohort2, open('infino-rcc/tcgakirc.precursor_to_stan_dicts.counts_and_tpm.pkl', 'wb'))

In [57]:
def make_test_dict_for_sample_ids(melted_dat, sampleids=None, metric='counts', processing='raw'):
    """
    melted_dat is melted_cohort2 from above or identical
    sampleids is list of samples to include in this cut. if None, include all samples
    metric must be counts or tpm
    processing must be raw or corrected
    """
    if not sampleids:
        test_df_for_dict = melted_dat.copy()
    else:
        test_df_for_dict = melted_dat[melted_dat['new_sample_id'].isin(sampleids)].copy()
    
    # reset sample id (otherwise stan complains/errors out)
    test_df_for_dict['new_sample_cat'] = test_df_for_dict['sample'].astype('category')
    test_df_for_dict['new_sample_id'] = test_df_for_dict['new_sample_cat'].cat.codes+1
    
    # return dict
    relevant_col_name = '%s_%s' % (metric, processing)
    assert relevant_col_name in test_df_for_dict.columns, 'invalid column specified'
    
    return {'N2': len(test_df_for_dict.index),
             'S2': len(test_df_for_dict.new_sample_id.unique()),
             'gene2': test_df_for_dict.new_gene_id.values,
             'sample2': test_df_for_dict.new_sample_id.values,
             'y2': test_df_for_dict[relevant_col_name].astype(int).values, # crucial to set as int (for negative binomial)
             'colname_test': relevant_col_name
           }

In [58]:
# since only 26, put them all together
assert make_test_dict_for_sample_ids(melted_cohort2)['S2'] == 26

for (metric, processing) in [('counts', 'raw'), ('tpm', 'raw'),
                             ('counts', 'corrected'), ('tpm', 'corrected')]:
    output_fname = 'infino-rcc/blahblah.standict.%s.%s.pkl' % (
        metric, processing)
    print(output_fname,)
    dict_to_write = make_test_dict_for_sample_ids(
            melted_cohort2,
            sampleids=None,
            metric=metric,
            processing=processing)
    print(dict_to_write['colname_test'])
    #pickle.dump(dict_to_write, open(output_fname, 'wb'))
    print()

AssertionError: 

Above cell is code from bladder cohort, where all unknown samples fit into one experiment.

We're not using that here, but keeping it around for future clones of this notebook.

Instead we'll cut into several experiments:

In [60]:
melted_cohort2.new_sample_id.describe()[['min', 'max']]

min     1.0
max    79.0
Name: new_sample_id, dtype: float64

In [61]:
cut_1 = list(range(1, 40))
#cut_2 = list(range(10, 20))
cut_2 = list(range(40, max(melted_cohort2.new_sample_id)+1))
print(cut_1,
      cut_2,
      #cut_3,
      len(cut_1),
      len(cut_2),
      #len(cut_3),
      len(cut_1) + len(cut_2) #+ len(cut_3)
     )

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39] [40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79] 39 40 79


In [62]:
for (sampleids, cutname) in zip([cut_1, cut_2], ['cut1', 'cut2']):
    for (metric, processing) in [('counts', 'raw'), ('tpm', 'raw'),
                                 ('counts', 'corrected'), ('tpm', 'corrected')]:
        output_fname = 'infino-rcc/tcgakirc.standict.%s.%s.%s.pkl' % (
            cutname, metric, processing)
        print(output_fname,)
        dict_to_write = make_test_dict_for_sample_ids(
                melted_cohort2,
                sampleids=sampleids,
                metric=metric,
                processing=processing)
        print(dict_to_write['colname_test'])
        print(dict_to_write['S2'])
        pickle.dump(dict_to_write, open(output_fname, 'wb'))
        print()

infino-rcc/tcgakirc.standict.cut1.counts.raw.pkl
counts_raw
39

infino-rcc/tcgakirc.standict.cut1.tpm.raw.pkl
tpm_raw
39

infino-rcc/tcgakirc.standict.cut1.counts.corrected.pkl
counts_corrected
39

infino-rcc/tcgakirc.standict.cut1.tpm.corrected.pkl
tpm_corrected
39

infino-rcc/tcgakirc.standict.cut2.counts.raw.pkl
counts_raw
40

infino-rcc/tcgakirc.standict.cut2.tpm.raw.pkl
tpm_raw
40

infino-rcc/tcgakirc.standict.cut2.counts.corrected.pkl
counts_corrected
40

infino-rcc/tcgakirc.standict.cut2.tpm.corrected.pkl
tpm_corrected
40



In [166]:
#pickle.dump(make_test_dict_for_sample_ids(melted_bladder2, cut_1), open('infino-rcc/bladder_dict_1.pkl', 'wb'))
#pickle.dump(make_test_dict_for_sample_ids(melted_bladder2, cut_2), open('infino-rcc/bladder_dict_2.pkl', 'wb'))

# Launch docker containers


```
cd infino-rcc

mkdir -p logs
chmod -R 777 logs

# run each of the below in a separate tmux:
```



In [69]:
for (sampleids, cutname) in zip([cut_1, cut_2], ['cut1', 'cut2']):
    for (metric, processing) in [('counts', 'raw'), ('tpm', 'raw'),
                                 ('counts', 'corrected'), ('tpm', 'corrected')]:
        training_dict_fname = 'tcgakirc.singleorigin.stan_data.%s_%s.pkl' % (metric, processing)
        cohort_dict_fname = 'tcgakirc.standict.%s.%s.%s.pkl' % (cutname, metric, processing)
        runname = 'tcgakirc_%s_%s_%s' % (cutname, metric, processing)
        print("""
        # {runname}: {training_dict_fname}, {cohort_dict_fname}
        mkdir -p logs/{runname} && chmod -R 777 logs/{runname} && docker run -it -d --name {runname} \\
        -v $HOME/immune-infiltrate-explorations:/home/jovyan/work \\
        hammerlab/infino-docker:latest \\
        bash -c "cd work/model-single-origin-samples/infino-rcc/ && python \\
        cohort_runstan.py {training_dict_fname} {cohort_dict_fname} {runname} 2>&1 | tee logs/{runname}/run_stan.{runname}.consoleout.txt"
    """.format(training_dict_fname=training_dict_fname, cohort_dict_fname=cohort_dict_fname, runname=runname))
        print("#" * 60)



        # tcgakirc_cut1_counts_raw: tcgakirc.singleorigin.stan_data.counts_raw.pkl, tcgakirc.standict.cut1.counts.raw.pkl
        mkdir -p logs/tcgakirc_cut1_counts_raw && chmod -R 777 logs/tcgakirc_cut1_counts_raw && docker run -it -d --name tcgakirc_cut1_counts_raw \
        -v $HOME/immune-infiltrate-explorations:/home/jovyan/work \
        hammerlab/infino-docker:latest \
        bash -c "cd work/model-single-origin-samples/infino-rcc/ && python \
        cohort_runstan.py tcgakirc.singleorigin.stan_data.counts_raw.pkl tcgakirc.standict.cut1.counts.raw.pkl tcgakirc_cut1_counts_raw 2>&1 | tee logs/tcgakirc_cut1_counts_raw/run_stan.tcgakirc_cut1_counts_raw.consoleout.txt"
    
############################################################

        # tcgakirc_cut1_tpm_raw: tcgakirc.singleorigin.stan_data.tpm_raw.pkl, tcgakirc.standict.cut1.tpm.raw.pkl
        mkdir -p logs/tcgakirc_cut1_tpm_raw && chmod -R 777 logs/tcgakirc_cut1_tpm_raw && docker run -it -d --name tcgakirc_cut1_tpm_raw

```
# monitor output with:
tail -f logs/tcgakirc*/*.consoleout.txt
```

### some scripts to make gcloud instances

In [75]:
for machine_id in range(1, 9):
    print("""

gcloud beta compute --project "pici-hammerlab" instances create "infino-{machine_id}" \\
--zone "us-east1-b" --machine-type "n1-highmem-4" --subnet "default" --no-restart-on-failure \\
--maintenance-policy "MIGRATE" \\
--service-account "infino@pici-hammerlab.iam.gserviceaccount.com" \\
--scopes "https://www.googleapis.com/auth/cloud-platform" \\
--min-cpu-platform "Automatic" \\
--image "ubuntu-1604-xenial-v20171026a" --image-project "ubuntu-os-cloud" \\
--boot-disk-size "60" --boot-disk-type "pd-standard" --boot-disk-device-name "infino-{machine_id}";

""".format(machine_id=machine_id))



gcloud beta compute --project "pici-hammerlab" instances create "infino-1" \
--zone "us-east1-b" --machine-type "n1-highmem-4" --subnet "default" --no-restart-on-failure \
--maintenance-policy "MIGRATE" \
--service-account "infino@pici-hammerlab.iam.gserviceaccount.com" \
--scopes "https://www.googleapis.com/auth/cloud-platform" \
--min-cpu-platform "Automatic" \
--image "ubuntu-1604-xenial-v20171026a" --image-project "ubuntu-os-cloud" \
--boot-disk-size "60" --boot-disk-type "pd-standard" --boot-disk-device-name "infino-1";




gcloud beta compute --project "pici-hammerlab" instances create "infino-2" \
--zone "us-east1-b" --machine-type "n1-highmem-4" --subnet "default" --no-restart-on-failure \
--maintenance-policy "MIGRATE" \
--service-account "infino@pici-hammerlab.iam.gserviceaccount.com" \
--scopes "https://www.googleapis.com/auth/cloud-platform" \
--min-cpu-platform "Automatic" \
--image "ubuntu-1604-xenial-v20171026a" --image-project "ubuntu-os-cloud" \
--boot-disk-size "60"